In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
from collections import defaultdict, Counter

In [30]:
def back_to_list(s):
    end = s.replace('[', '').replace(']', '').replace('\'', '').replace(' ', '').split(',')
    return end

def load_csv(filename):
    df = pd.read_csv(os.path.join('..', 'data', filename))
    df.body = df.body.apply(lambda x: back_to_list(x))
    df.title = df.title.apply(lambda x: back_to_list(x))
    return df

def generate_vocab(df, include_headline=True, include_body=False):
    vocab = set()
    for row in df.title:
        vocab = vocab.union(set(row))
    if include_body:
        for row in df.body:
            vocab = vocab.union(set(row))       
    vocab = list(vocab)
    print(f'Vocab Size: {len(vocab)}')
    return vocab

def p_w_H_given_D(df, vocab):
    # denominator, numerator
    p_w_in_D, p_w_in_HD = {}, {}
    for word in tqdm(vocab):
        dcount, hdcount = 1, 1
        for article in df.iterrows():
            if word in article[1].body:
                dcount += 1
                if word in article[1].title:
                    hdcount += 1
        p_w_in_D[word] = dcount
        p_w_in_HD[word] = hdcount
    return {word: p_w_in_HD[word] / p_w_in_D[word] for word in vocab}

def length_prob(df):
    length_list = [len(headline) for headline in df.title]
    length_counter = Counter(length_list)
    return {length: count/len(length_list) for length, count in length_counter.items()}

def freq_to_prob(whole, item):
    total = sum(whole[item].values())
    whole[item] = {k: v / total for k, v in whole[item].items()}
        
def bigramLM(df):
    transition = defaultdict(dict)
    for line in tqdm(df.body): 
        prev_word = None
        for word in line:
            if not prev_word:
                prev_word = 'BOS'
            transition[prev_word][word] = transition[prev_word].get(word, 1)
            prev_word = word
        transition[prev_word]['EOS'] = transition[prev_word].get('EOS', 1)   
        
    for line in tqdm(df.title): 
        prev_word = None
        for word in line:
            if not prev_word:
                prev_word = 'BOS'
            transition[prev_word][word] = transition[prev_word].get(word, 1)
            prev_word = word
        transition[prev_word]['EOS'] = transition[prev_word].get('EOS', 1) 
        
    _ = list(map(lambda item: freq_to_prob(transition, item), transition))
    # return transposed dataframe so that each row index represent the prev_words
    return pd.DataFrame(transition).T.fillna(0)

### Sample run with train[:2]

In [3]:
df = load_csv('reuters_train.csv')[:1000]

In [4]:
vocab = generate_vocab(df, include_body=True)

Vocab Size: 13530


In [5]:
H_given_D = p_w_H_given_D(df, vocab)

100%|████████████████████████████████████████████████████████████████████████████| 13530/13530 [44:40<00:00,  5.05it/s]


In [6]:
length_prob(df)

{7: 0.34, 8: 0.198, 5: 0.11, 6: 0.258, 9: 0.063, 4: 0.024, 10: 0.007}

In [31]:
T = bigramLM(df)

100%|███████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 83536.90it/s]


In [32]:
T.loc['millions',:][T.loc['millions',:]!= 0.0]

in      0.333333
of      0.333333
dart    0.333333
Name: millions, dtype: float64

### Viterbi Decoding

In [65]:
def Viterbi(doc, headline_len=6, H_given_D=H_given_D, transition=T):
    # Generate vocab for headline candidates words
    vocab = list(set(doc))
    # F to store probabilities and B to store backpointers
    F = pd.DataFrame(columns=range(headline_len), index=vocab).fillna(0)
    B = pd.DataFrame(columns=range(headline_len), index=vocab).fillna(-1)
    # Forward Path
    for word in vocab:
        F.loc[word, 0] = H_given_D.get(word, 1e-5) * T.loc['BOS', word]
    for t in range(1, headline_len):
        for i, word in enumerate(vocab):
            max_prob, max_index = -1, -1
            for j, prev_word in enumerate(vocab):
                prob = H_given_D.get(word, 1e-5) * T.loc[prev_word, word] * F.loc[prev_word, t-1]
                if prob > max_prob:
                    max_prob, max_index = prob, j
                    F.iloc[i, t] = max_prob
                    B.iloc[i, t] = max_index
    # Backtracking
    pointer = np.argmax(F.iloc[:,-1])
    pos_seq = [vocab[pointer]]
    for i in range(headline_len-1, 0, -1):
        pointer = B.iloc[pointer, i]
        pos_seq.insert(0, vocab[pointer])
    return pos_seq

In [64]:
for doc in df.body[:1]:
    print(Viterbi(doc, 5))

['sapporo', 'breweries', 'ltd', 'year', 'notes']


In [67]:
for doc in df.body[:1]:
    print(Viterbi(doc, ))

['sapporo', 'breweries', 'ltd', 'and', '10014', 'issue']


In [71]:
def Viterbi_log(doc, headline_len=6, H_given_D=H_given_D, transition=T):
    # Generate vocab for headline candidates words
    vocab = list(set(doc))
    # F to store probabilities and B to store backpointers
    F = pd.DataFrame(columns=range(headline_len), index=vocab).fillna(0)
    B = pd.DataFrame(columns=range(headline_len), index=vocab).fillna(-1)
    # Forward Path
    for word in vocab:
        F.loc[word, 0] = np.log(H_given_D.get(word, 1e-5)) + np.log(T.loc['BOS', word]+1e-5)
    for t in range(1, headline_len):
        for i, word in enumerate(vocab):
            max_prob, max_index = float('-inf'), -1
            for j, prev_word in enumerate(vocab):
                prob = np.log(H_given_D.get(word, 1e-5)) + np.log(T.loc[prev_word, word]+1e-5) \
                + np.log(F.loc[prev_word, t-1]+1e-5)
                if prob > max_prob:
                    max_prob, max_index = prob, j
                    F.iloc[i, t] = max_prob
                    B.iloc[i, t] = max_index
    # Backtracking
    pointer = np.argmax(F.iloc[:,-1])
    pos_seq = [vocab[pointer]]
    for i in range(headline_len-1, 0, -1):
        pointer = B.iloc[pointer, i]
        pos_seq.insert(0, vocab[pointer])
    return pos_seq

In [72]:
for doc in df.body[:1]:
    print(Viterbi_log(doc, ))

<ipython-input-71-3faf3e6957a8>:15: RuntimeWarning: invalid value encountered in log
  + np.log(F.loc[prev_word, t-1]+1e-5)


['corp', 'bank', 'corp', 'bank', 'corp', '\\x03']
